In [2]:
import numpy as np
import pandas as pd

train_df = pd.read_csv('dataset/train.csv')
test_df = pd.read_csv('dataset/test.csv')

In [52]:
train_df.head()

,id,original_text,lang,retweet_count,original_author,sentiment_class,clean_tweet,tokens
0,1.245025e+18,Happy #MothersDay to all you amazing mothers o...,en,0,BeenXXPired,0,happi mothersday amaz mother know hard not abl...,NC
1,1.245759e+18,Happy Mothers Day Mum - I'm sorry I can't be t...,en,1,FestiveFeeling,0,happi mother day mum im sorri cant bring mothe...,NC
2,1.246087e+18,Happy mothers day To all This doing a mothers ...,en,0,KrisAllenSak,-1,happi mother day mother day work today quiet t...,NC
3,1.244803e+18,Happy mothers day to this beautiful woman...ro...,en,0,Queenuchee,0,happi mother day beauti womanroyalti sooth mum...,NC
4,1.244876e+18,Remembering the 3 most amazing ladies who made...,en,0,brittan17446794,-1,rememb 3 amaz ladi made late grandmoth iri mum...,NC


In [64]:
from nltk.tokenize import TweetTokenizer # a tweet tokenizer from nltk.
tokenizer = TweetTokenizer()
def tokenize(tweet):

    tweet = str(tweet.lower())
    tokens = tokenizer.tokenize(tweet)
    tokens = list(filter(lambda t: not t.startswith('@'), tokens))
    tokens = list(filter(lambda t: not t.startswith('#'), tokens))
    tokens = list(filter(lambda t: not t.startswith('http'), tokens))
    return tokens

    
def postprocess(data):
    data['tokens'] = data['original_text'].apply(tokenize)  ## progress_map is a variant of the map function plus a progress bar. Handy to monitor DataFrame creations.
    print
    data = data[data.tokens != 'NC']
    return data

data = postprocess(train_df)

In [65]:
data.head()

,id,original_text,lang,retweet_count,original_author,sentiment_class,clean_tweet,tokens
0,1.245025e+18,Happy #MothersDay to all you amazing mothers o...,en,0,BeenXXPired,0,happi mothersday amaz mother know hard not abl...,"[happy, to, all, you, amazing, mothers, out, t..."
1,1.245759e+18,Happy Mothers Day Mum - I'm sorry I can't be t...,en,1,FestiveFeeling,0,happi mother day mum im sorri cant bring mothe...,"[happy, mothers, day, mum, -, i'm, sorry, i, c..."
2,1.246087e+18,Happy mothers day To all This doing a mothers ...,en,0,KrisAllenSak,-1,happi mother day mother day work today quiet t...,"[happy, mothers, day, to, all, this, doing, a,..."
3,1.244803e+18,Happy mothers day to this beautiful woman...ro...,en,0,Queenuchee,0,happi mother day beauti womanroyalti sooth mum...,"[happy, mothers, day, to, this, beautiful, wom..."
4,1.244876e+18,Remembering the 3 most amazing ladies who made...,en,0,brittan17446794,-1,rememb 3 amaz ladi made late grandmoth iri mum...,"[remembering, the, 3, most, amazing, ladies, w..."


In [66]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(np.array(data['tokens']),
                                                    np.array(data['sentiment_class']), test_size=0.2)



In [67]:
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
LabeledSentence = doc2vec.LabeledSentence

def labelizeTweets(tweets, label_type):
    labelized = []
    for i,v in tqdm(enumerate(tweets)):
        label = '%s_%s'%(label_type,i)
        labelized.append(LabeledSentence(v, [label]))
    return labelized

x_train = labelizeTweets(x_train, 'TRAIN')
x_val = labelizeTweets(x_val, 'VAL')

0it [00:00, ?it/s]/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  if __name__ == '__main__':
2588it [00:00, 128691.36it/s]
647it [00:00, 143173.72it/s]


In [68]:
x_train[0]

LabeledSentence(words=['happy', 'mothers', 'day', 'to', 'the', 'many', 'mums', 'we', 'support', 'it', 'might', 'be', 'a', 'tough', 'few', 'weeks', 'ahead', 'but', 'you', 'can', 'do', 'this', ':/', '/', 'twitter.com/whyouthagency/', 'status', '/', '12417638639', '2369152', '1', '…'], tags=['TRAIN_0'])

In [69]:
n_dim = 200
tweet_w2v = Word2Vec(size=n_dim, min_count=10)
tweet_w2v.build_vocab([x.words for x in tqdm(x_train)])
tweet_w2v.train([x.words for x in tqdm(x_train)], total_examples=len(x_train), epochs=10)

100%|██████████| 2588/2588 [00:00<00:00, 1201024.42it/s]


(460719, 953740)

In [71]:
tweet_w2v.most_similar('happy')

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('day', 0.9567115306854248),
 ('beautiful', 0.9230166673660278),
 ('every', 0.9017370939254761),
 ('wishing', 0.8838872909545898),
 ('heavenly', 0.8785157203674316),
 ('belated', 0.876350462436676),
 ('mummy', 0.8758375644683838),
 ('fabulous', 0.8655744791030884),
 ("'", 0.8653419017791748),
 ('figures', 0.857086181640625)]

In [75]:
import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.plotting import figure, show, output_notebook

# defining the chart
output_notebook()
plot_tfidf = bp.figure(plot_width=700, plot_height=600, title="A map of 10000 word vectors",
    tools="pan,wheel_zoom,box_zoom,reset,hover",
    x_axis_type=None, y_axis_type=None, min_border=1)

# getting a list of word vectors. limit to 10000. each is of 200 dimensions
word_vectors = [tweet_w2v[w] for w in list(tweet_w2v.wv.vocab.keys())[:5000]]

# dimensionality reduction. converting the vectors to 2d vectors
from sklearn.manifold import TSNE
tsne_model = TSNE(n_components=2, verbose=1, random_state=0)
tsne_w2v = tsne_model.fit_transform(word_vectors)

# putting everything in a dataframe
tsne_df = pd.DataFrame(tsne_w2v, columns=['x', 'y'])
tsne_df['words'] = list(tweet_w2v.wv.vocab.keys())[:5000]

# plotting. the corresponding word appears when you hover on the data point.
plot_tfidf.scatter(x='x', y='y', source=tsne_df)
hover = plot_tfidf.select(dict(type=HoverTool))
hover.tooltips={"word": "@words"}
show(plot_tfidf)

Loading BokehJS ...

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':


[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 756 samples in 0.010s...
[t-SNE] Computed neighbors for 756 samples in 0.509s...
[t-SNE] Computed conditional probabilities for sample 756 / 756
[t-SNE] Mean sigma: 0.190022
[t-SNE] KL divergence after 250 iterations with early exaggeration: 65.121391
[t-SNE] KL divergence after 1000 iterations: 0.897207


In [77]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(analyzer=lambda x: x, min_df=10)
matrix = vectorizer.fit_transform([x.words for x in x_train])
tfidf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))
print('vocab size :', len(tfidf))

vocab size : 735


In [78]:
def buildWordVector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += tweet_w2v[word].reshape((1, size)) * tfidf[word]
            count += 1.
        except KeyError: # handling the case where the token is not
                         # in the corpus. useful for testing.
            continue
    if count != 0:
        vec /= count
    return vec

In [79]:
from sklearn.preprocessing import scale
train_vecs_w2v = np.concatenate([buildWordVector(z, n_dim) for z in tqdm(map(lambda x: x.words, x_train))])
train_vecs_w2v = scale(train_vecs_w2v)

val_vecs_w2v = np.concatenate([buildWordVector(z, n_dim) for z in tqdm(map(lambda x: x.words, x_val))])
val_vecs_w2v = scale(val_vecs_w2v)

0it [00:00, ?it/s]/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
2588it [00:01, 1326.53it/s]
0it [00:00, ?it/s]/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
647it [00:00, 1452.97it/s]


In [144]:
y_train

array([-1, -1,  0, ...,  0,  0,  0])

In [145]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import InverseTimeDecay
from tensorflow.keras.callbacks import EarlyStopping
from keras.utils import to_categorical

model = Sequential()
model.add(Dense(32, activation='relu', input_dim=200))
model.add(Dense(3, activation='softmax'))
model.compile( optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

model.fit(train_vecs_w2v, to_categorical(y_train, 3), epochs=9, batch_size=32, verbose=2)

Train on 2588 samples
Epoch 1/9
2588/2588 - 1s - loss: 1.1046 - accuracy: 0.4660
Epoch 2/9
2588/2588 - 0s - loss: 1.0441 - accuracy: 0.5104
Epoch 3/9
2588/2588 - 0s - loss: 1.0327 - accuracy: 0.5147
Epoch 4/9
2588/2588 - 0s - loss: 1.0329 - accuracy: 0.5158
Epoch 5/9
2588/2588 - 0s - loss: 1.0260 - accuracy: 0.5220
Epoch 6/9
2588/2588 - 0s - loss: 1.0248 - accuracy: 0.5216
Epoch 7/9
2588/2588 - 0s - loss: 1.0232 - accuracy: 0.5185
Epoch 8/9
2588/2588 - 0s - loss: 1.0177 - accuracy: 0.5205
Epoch 9/9
2588/2588 - 0s - loss: 1.0176 - accuracy: 0.5209


In [147]:
y_val

array([-1, -1, -1, -1,  0,  0, -1,  0,  1, -1, -1, -1,  1,  0,  0,  0,  0,
        1,  1,  0,  1,  0, -1,  0, -1,  1,  1,  0, -1,  0,  0,  0,  0,  0,
        0,  1,  0,  0,  0,  0,  0,  1,  0,  0, -1,  0,  0,  0, -1, -1,  0,
        1,  0,  0, -1, -1,  0, -1,  0,  0, -1,  1, -1,  0,  0,  1,  1,  0,
        0,  0,  0, -1,  0,  1,  0,  1,  1,  0,  1,  0,  0,  1,  1, -1,  1,
       -1,  1,  0,  0,  0,  0,  0,  0, -1, -1,  0,  0,  1,  0, -1,  1, -1,
        0,  1,  0,  0,  0,  0, -1,  0, -1,  1,  1,  0,  1,  1,  1, -1, -1,
       -1,  0, -1, -1,  0,  1,  0,  0,  0,  1, -1,  1,  1,  0,  0,  0, -1,
       -1,  1,  0,  0, -1,  1, -1, -1,  0,  0,  0,  0,  1, -1,  0,  0,  0,
       -1,  1, -1,  0, -1,  1, -1,  0, -1,  0,  0,  0,  1,  1,  1,  0,  0,
        1,  1,  0,  0, -1, -1, -1,  0, -1,  1,  0,  0,  0,  0,  0,  1,  0,
        0,  0,  0,  1,  1, -1,  0, -1,  0,  1,  0,  1,  0,  0, -1,  1,  1,
        1,  0,  0,  0, -1,  0,  1, -1,  0,  1,  0,  0,  0,  0,  0,  0,  1,
        0,  1,  0, -1,  0

In [148]:
score = model.evaluate(val_vecs_w2v, to_categorical(y_val, 3), batch_size=128, verbose=2)

647/647 - 0s - loss: 1.0768 - accuracy: 0.5178


In [149]:
test_data = postprocess(test_df)

In [150]:
test_data.head()

,id,original_text,lang,retweet_count,original_author,clean_tweet,tokens
0,1.246628e+18,"3. Yeah, I once cooked potatoes when I was 3 y...",en,0,LToddWood,3 yeah cook potato 3 year old mean threw bag s...,"[3, ., yeah, ,, i, once, cooked, potatoes, whe..."
1,1.245898e+18,"Happy Mother's Day to all the mums, step-mums,...",en,0,iiarushii,happi mother day mum stepmum grandmoth dad cov...,"[happy, mother's, day, to, all, the, mums, ,, ..."
2,1.244717e+18,"I love the people from the UK, however, when I...",en,0,andreaanderegg,love peopl uk howev awok saw mothersday trend ...,"[i, love, the, people, from, the, uk, ,, howev..."
3,1.245730e+18,Happy 81st Birthday Happy Mother’s Day to my m...,en,1,TheBookTweeters,happi 81st birthday happi mother day mum can't...,"[happy, 81st, birthday, happy, mother, ’, s, d..."
4,1.244636e+18,Happy Mothers day to all those wonderful mothe...,en,0,andreaanderegg,happi mother day wonder mother world live pres...,"[happy, mothers, day, to, all, those, wonderfu..."


In [151]:
x_test = np.array(test_data['tokens'])

In [152]:
x_test = labelizeTweets(x_test, 'TEST')

0it [00:00, ?it/s]/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  if __name__ == '__main__':
1387it [00:00, 131356.12it/s]


In [153]:
x_test[0]

LabeledSentence(words=['3', '.', 'yeah', ',', 'i', 'once', 'cooked', 'potatoes', 'when', 'i', 'was', '3', 'years', 'old', ',', 'and', 'by', 'that', 'i', 'mean', 'i', 'threw', 'a', 'bag', 'of', 'spuds', 'into', 'the', 'toilet', '.', '"', 'happy', 'mothers', 'day', ',', 'i', 'made', 'breakfast', '!', '"', '66', '.', 'that', 'time', 'i', 'thought', 'i', 'was', 'cool', 'by', 'drawing', 'the', 'naked', 'lady', 'dachshund', 'on', 'the', 'overhead', 'projector', 'in', 'psychology', 'class', '.', '8'], tags=['TEST_0'])

In [154]:
n_dim = 200
tweet_w2v_test = Word2Vec(size=n_dim, min_count=10)
tweet_w2v_test.build_vocab([x.words for x in tqdm(x_test)])
tweet_w2v_test.train([x.words for x in tqdm(x_test)], total_examples=len(x_test), epochs=10)

100%|██████████| 1387/1387 [00:00<00:00, 469531.85it/s]


(230348, 517580)

In [155]:
test_vecs_w2v = np.concatenate([buildWordVector(z, n_dim) for z in tqdm(map(lambda x: x.words, x_test))])
test_vecs_w2v = scale(test_vecs_w2v)

0it [00:00, ?it/s]/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
1387it [00:01, 1217.53it/s]


In [156]:
test_predictions = model.predict(test_vecs_w2v)

In [141]:
test_predictions[1]

array([0.43673107, 0.36702523, 0.19624373], dtype=float32)

In [157]:
label = [np.argmax(i) for i in test_predictions]
class_label = [x for x in label]
print(class_label[:3])
submission = pd.DataFrame({ 'id': test_df.id, 'sentiment_class': class_label })
submission.to_csv('test0050.csv', index=False)

[0, 2, 0]


In [160]:
df_test = pd.read_csv('test0050.csv')
def replace_two(n):
    if n == 2:
        return -1
    else:
        return n
df_test['sentiment_class'] = df_test['sentiment_class'].apply(replace_two)

In [161]:
df_test.to_csv('test_0058.csv', index=False)